# Three kinds of Implicits and Type Classes in Scala

Implicit parameters are useful in a number of situations and can remove a lot of repetitive code.  But they can be over used as well, leading to magical and hard to understand code.  The Scala standard library makes quite a bit of use of these.

### ➤ * Implicit Parameters *

Suppose you want a function that joins two strings together.

In [ ]:
def ipExample1(a: String, b: String): String = 
    List(a, b).mkString("")

ipExample1("foo", "bar")

You can split the parameters into a parameter listas such.  Each list of parameters is called a parameter group.

In [ ]:
def ipExample2(a: String)(b: String): String = 
    List(a, b).mkString("")

ipExample2("foo")("bar")

You can create a partially applied function (aka lambda) by passing only the first argument.  Then call that with the second argument.  In functional programming, this is called currying.

In [ ]:
val ipPartial = ipExample2("foo")(_)
ipPartial("bar")

You can mark a parameter group within the parameter list as implicit.  This applies to all the parameters within that group.  When calling this function, all the non-implicit parameters must be supplied with arguments, as usual.  The implicit parameters can be supplied with arguments, if you want, but it is no longer required.  Instead, it will search for a variable, marked itself as implicit, that is in scope and of the matching type, for each of the parameters in the group.  If there are zero or more than one implicit variables in scope of the matching type, the code will not compile.

In [ ]:
def ipExample3(a: String)(implicit b: String): String = 
    List(a, b).mkString("")

implicit val bar: String = "bar"

ipExample3("foo")
ipExample3("foo")("zzz")

Implicit objects can also be used for implicit parameters.  In general, objects can be used much like values in Scala.  This is useful when you want to be able to specify a group of behaviors to be used by the function.  One example of this usage is JSON4s.  Most functions ........

In [ ]:
trait IPFormatter {
    def format(string: String): String
}

implicit object IPFormatter1 extends IPFormatter {
    def format(string: String): String = 
        "--- %s ---".format(string)
}

object IPFormatter2 extends IPFormatter {
    def format(string: String): String = 
        "+++ %s +++".format(string)
}

In [ ]:
def ipExample4(string: String)(implicit formatter: IPFormatter): String =
  formatter.format(string)

ipExample4("foobar")
ipExample4("foobar")(IPFormatter2)

This is useful when you want to be able to specify a group of behaviors to be used by the function. One example of this usage is JSON4s. Most functions in this library have an implicit parameter group that takes a single argument of type `Format`.  You can use the defaults by using an import.  Or you can specify your own.

Another example might be something like logging.  You can define a logging service in your `main` as implicit.  Then any function that wants to log can simply have an implicit parameter of the right type and import the definition.  But for testing purposes, you can specify a logging service that holds the logs, which can then be queries by the tests.

The other way to do this is to pass these services down the entire call path as needed.  But that gets kind of tiring.  With implicits, all you need is the import.

### ➤ * Implicit Type Conversion *

In your code, if some type B is required, but you have a type A, the compiler will look for a single implicit function in scope that takes an A and returns a B.  Note, any implicit functions within the companion object of either the source or destination type are automatically in scope.

In [ ]:
object ITCExample1 {
  implicit def convert5to4(x: ITCExample2): ITCExample1 = {
    val Array(firstName, lastName) = x.fullName.split(" ")
    ITCExample1(firstName, lastName)
  }
    
  implicit def convert(x: ITCExample1): ITCExample2 = {
    val fullName = List(x.firstName, x.lastName).mkString(" ")
    ITCExample2(fullName)
  }
}

case class ITCExample1(firstName: String, lastName: String)

case class ITCExample2(fullName: String)

In [ ]:
val a: ITCExample1 = ITCExample1("Bob", "Smith")
val b: ITCExample2 = a
val c: ITCExample1 = b

Several types have implicit conversions to other types defined in their companion object.  For example the `Int` type has an implicit conversion to a `Double` __[here](https://github.com/scala/scala/blob/v2.12.8/src/library/scala/Int.scala#L482)__.  That allows the developer to use an `Int` anywhere a `Double` is required without any explicit transform.  The designers felt this automatic conversion was okay, and useful, since no information is lost. Note there is no implicit conversion from a `Double` to an `Int`, as this would effectively lose information.  It is up to the developer to explicitly chose how they want to do that transform.

In [ ]:
val d: Int = 123
val e: Double = d

### ➤ * Implicit Methods *

Implicit methods are very similar in concept to implicit type conversion.  Consider calling the method `distinct` on a `String`.

In [ ]:
"aaabbbccc".distinct

In Scala, the `String` type is an alias for the Java `String`.  And the `String` class in Java does not have a `distinct` method as a member.  So what is going on here?  Where did this method come from?  It came from the `StringOps` class in Scala.

If you have a variable of type A and try to call a method that is not a member of the class, it will try to find a single implicit function within scope that converts the variable from type A to any type B which does have the method as a member.  If there are zero or more than one type conversion functions within scope which meet this criteria, the code will not compile.

Within the `Predef` object, which is automatically imported into every Scala file, there is an implicit function called `augmentString` __[here](https://github.com/scala/scala/blob/v2.12.8/src/library/scala/Predef.scala#L374)__, which converts a `String` to a `StringOps`.  Thus the `String` is automatically converted into a `StringOps` because it has the `distinct` method as a member.

In [1]:
case class MyStringOps1(string: String) {
  def multiply1(x: Int): String = (1 to x).map(_ => string).mkString("")
}

implicit def convert1(string: String): MyStringOps1 = MyStringOps1(string)

"hello".multiply1(3)

defined class MyStringOps1
defined function convert1
res0_2: String = "hellohellohello"

The class and implicit function can be combined into a single implicit class.  In this form, the class can not be a case class, and must take a single non-implicit parameter.  There can also be no other entities in scope with the same name.

In [2]:
implicit class MyStringOps2(string: String) {
    def multiply2(x: Int): String = (1 to x).map(_ => string).mkString("")
}

"hello".multiply2(3)

defined class MyStringOps2
res1_1: String = "hellohellohello"

This kind of implicit and it's behavior is the foundation required for type classes described next.

### ➤ * Type Classes *

Type classes are an ad-hoc method of adding behaviors to a set of types, without having to change them.  Here is an __[article](https://blog.scalac.io/2017/04/19/typeclasses-in-scala.html)__ describing them in detail for Scala.  The concept came from __[Haskell](http://learnyouahaskell.com/types-and-typeclasses#typeclasses-101)__.  They are also, in turn, the foundation for functional programming libraries such as __[Cats](https://typelevel.org/cats/)__ and __[Scalaz](https://scalaz.github.io/)__.

The first step is to create a `trait` which defines the methods and type signatures for the desired behaviors.

In [ ]:
trait MyTypeClass[A] {
  def multiply3(x: Int): A
}

The next step is to create implicit type conversion functions from each of the type you want to have this set of behaviors to an annonymous implementation of the trait.

In [ ]:
implicit def convert2(string: String) = new MyTypeClass[String] {
  def multiply3(x: Int): String = (1 to x).map(_ => string).mkString("")
}

implicit def convert3(int: Int) = new MyTypeClass[Int] {
  def multiply3(x: Int): Int = (1 to x).map(_ => int.toString).mkString("").toInt
}

Then you can simply call any of the trait methods on any of the types belonging to the type class.

In [ ]:
"hello".multiply3(3)
123.multiply3(3)